Interpolate solution over a "whole-cell" mesh. Here, we illustrate this using the stored solution for 2.8 micron indentation of the nuclear (alpha0 = 5), with the data supplied on Zenodo. This can readily be altered to other simulation cases, provided the mesh definition is adjusted accordingly.

In [ ]:
import dolfin as d
import smart
import h5py
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tkinter as tk
import os, sys
import re
from pathlib import Path
cur_dir = str(Path.cwd() / "..")
sys.path.append(f"{cur_dir}/utils")
import spread_cell_mesh_generation as mesh_gen

mesh_file = f"{cur_dir}/meshes/nanopillars_indent/nanopillars_indent2.8/spreadCell_mesh.h5"
ind_files = False
display = False
num_saved_pts = 501
same_mesh = False
folder = "simulation_results_2.8indent"
results_path = f"{cur_dir}/analysis_data/{folder}"
# if "pore_size0.2" in folder and "rate1" in folder and "transport100" in folder:
#     print("Process this one!")
# else:
#     continue
# if "interp" in folder or not Path(f"{parent_path}/{folder}").is_dir() or folder=="__pycache__":
#     continue
interp_path = f"{cur_dir}/analysis_data/{folder}_interp"
parent_mesh = smart.mesh.ParentMesh(
    mesh_filename=mesh_file,
    mesh_filetype="hdf5",
    name="parent_mesh",)

comm = parent_mesh.mpi_comm
dmesh = parent_mesh.dolfin_mesh
dim = parent_mesh.dimensionality
mf_cell = d.MeshFunction("size_t", dmesh, dim, value=0)
mf_facet = d.MeshFunction("size_t", dmesh, dim-1, value=0)
hdf5 = d.HDF5File(comm, mesh_file, "r")
hdf5.read(mf_cell, f"/mf{dim}")
hdf5.read(mf_facet, f"/mf{dim-1}")
hdf5.close()

# parse mesh info from folder name
height_str = re.findall(r"nanopillars_h.*?_", folder)
if height_str == []:
    # then an indentation style mesh
    rad = 0.5
    height = 3.0
    pitch = 3.5
    indent_str = re.findall(r"indent.*?_", folder)
    if len(indent_str) == 0:
        indent_str = re.findall(r"_.*?indent", folder)
        indentation = float(indent_str[0][-9:-6])
    else:
        indentation = float(indent_str[0][6:-1])
    contactRad = 15.5
else:
    height = float(height_str[0][13:-1])
    pitch_str = re.findall(r"nanopillars_h.*?_p.*?_", folder)
    pitch = float(pitch_str[0][len(height_str[0])+1:-1])
    rad_str = re.findall(r"nanopillars_h.*?_p.*?_r.*?_", folder)
    rad = float(rad_str[0][len(pitch_str[0])+1:-1])
    contactRad_str = re.findall(r"cellRad.*?_", folder)
    if contactRad_str == []:
        contactRad_str = re.findall(r"cellRad.*", folder)
        contactRad = float(contactRad_str[0][7:])
    else:
        contactRad = float(contactRad_str[0][7:-1])
    indentation = 0

hEdge = 1.0
hNP = hEdge * 0.5
nanopillars = [rad, height, pitch]
if same_mesh:
    cell_mesh = dmesh
    facet_markers = mf_facet
    cell_markers = mf_cell
else:
    cell_mesh, facet_markers, cell_markers, substrate_markers, curv_markers, u_nuc, a_nuc = mesh_gen.create_3dcell(
                                                                contactRad=contactRad,
                                                                hEdge=hEdge, hInnerEdge=hEdge,
                                                                hNP=hNP,
                                                                nanopillars=nanopillars,
                                                                return_curvature=True,
                                                                sym_fraction=1,
                                                                nuc_compression=indentation,
                                                                use_tmp=False)
results_file_list = []
tVec = []
for file in os.listdir(results_path):
    # check the files which end with specific extension
    if file.endswith(".h5"):
        results_file_list.append(file)
    if ind_files:
        if file.endswith(".xdmf"):
            xdmf_file = open(f"{results_path}/{file}", "r")
            xdmf_string = xdmf_file.read()
            found_pattern = re.findall(r"Time Value=\"?[^\s]+", xdmf_string)
            for i in range(len(found_pattern)):
                tVec.append(float(found_pattern[i][12:-1]))
    else:
        if file.endswith(".xdmf") and tVec==[]:
            xdmf_file = open(f"{results_path}/{file}", "r")
            xdmf_string = xdmf_file.read()
            found_pattern = re.findall(r"Time Value=\"?[^\s]+", xdmf_string)
            for i in range(len(found_pattern)):
                tVec.append(float(found_pattern[i][12:-1]))

if ind_files:
    tVec = np.array(tVec)
    tVec = np.unique(tVec)
    tVec = np.sort(tVec)
    tVec = list(tVec)

cell_vals = np.unique(mf_cell.array())
cell_vals = cell_vals[np.logical_and(cell_vals !=0, cell_vals < 1e9)]
facet_vals = np.unique(mf_facet.array())
facet_vals = facet_vals[np.logical_and(facet_vals !=0, facet_vals < 1e9)]
child_meshes = []
child_mesh_len = []
for i in range(len(cell_vals)+len(facet_vals)):
    if i < len(cell_vals):
        mesh = d.MeshView.create(mf_cell, cell_vals[i])
    else:
        mesh = d.MeshView.create(mf_facet, facet_vals[i-len(cell_vals)])
    child_meshes.append(mesh)
    child_mesh_len.append(len(mesh.coordinates()))

for j in range(len(results_file_list)):
    cur_file = h5py.File(f"{results_path}/{results_file_list[j]}", "r")
    try:
        test_array = cur_file["VisualisationVector"]["0"][:]
    except:
        print("error")
        continue
    find_mesh = len(test_array) == np.array(child_mesh_len)
    if len(np.nonzero(find_mesh)[0]) != 1:
        raise ValueError("Could not identify submesh")
    else:
        submesh_idx = np.nonzero(find_mesh)[0][0]
        cur_mesh = child_meshes[submesh_idx]
        if submesh_idx < len(cell_vals):
            marker_cur = cell_vals[submesh_idx]
        else:
            marker_cur = facet_vals[submesh_idx-len(cell_vals)]
    
    subdomain_mf = d.MeshFunction("size_t", cur_mesh, cur_mesh.topology().dim(), 1)
    
    Vcur = d.FunctionSpace(cur_mesh, "P", 1)
    dvec = d.Function(Vcur)
    time_pt_tags = cur_file["VisualisationVector"].keys()
    num_time_points = len(time_pt_tags)
    tNew = np.linspace(min(tVec), 10000, num_saved_pts)#max(tVec), num_saved_pts)
    dof_map = d.dof_to_vertex_map(Vcur)[:]
    
    if not same_mesh:
        if marker_cur in facet_markers.array():
            cur_mesh_new = d.MeshView.create(facet_markers, marker_cur)
        else:
            cur_mesh_new = d.MeshView.create(cell_markers, marker_cur)
        Vnew = d.FunctionSpace(cur_mesh_new, "P", 1)
        dvec_new = d.Function(Vnew)
    else:
        Vnew = Vcur
        dvec_new = d.Function(Vcur)

    results_cur = d.XDMFFile(
        comm, str(f"{interp_path}/{results_file_list[j][0:-3]}_interp.xdmf")
    )
    results_cur.parameters["flush_output"] = True

    for i in range(num_saved_pts):
        try:
            if i==0:
                cur_array = cur_file["VisualisationVector"]["0"][:]
            elif tNew[i] == tVec[-1]:
                cur_array = cur_file["VisualisationVector"][str(num_time_points-1)][:]
            else:
                tCur = tNew[i]
                tVec_np = np.array(tVec)
                beforeLogic = tVec_np <= tCur
                beforeIdx = np.nonzero(beforeLogic)[0][-1]
                tBefore = tVec_np[beforeIdx]
                afterLogic = tVec_np > tCur
                afterIdx = np.nonzero(afterLogic)[0][0]
                tAfter = tVec_np[afterIdx]
                interpFactor = (tCur - tBefore) / (tAfter - tBefore)
                cur_array_before = cur_file["VisualisationVector"][str(beforeIdx)][:]
                cur_array_after = cur_file["VisualisationVector"][str(afterIdx)][:]
                cur_array = cur_array_before + interpFactor*(cur_array_after - cur_array_before)
        except:
            print('error')
            continue
        # array matches mesh ordering; reorder according to dof mapping for Vcur
        cur_array = cur_array[dof_map]
        orig_coords = Vcur.tabulate_dof_coordinates()

        dvec.vector().set_local(cur_array)
        dvec.vector().apply("insert")
        dvec.set_allow_extrapolation(True)

        # now interpolate onto new mesh
        new_coords = Vnew.tabulate_dof_coordinates()
        symm = 1/8
        if len(new_coords)==len(orig_coords):
            new_array = cur_array
        else:
            new_array = []
            for k in range(len(new_coords)):
                if symm < 1:
                    x_cur, y_cur, z_cur = new_coords[k]
                    thetaCur = np.arctan2(y_cur, x_cur)
                    r_cur = np.sqrt(x_cur**2 + y_cur**2)
                    thetaSymm = thetaCur % (2*np.pi*symm*2)
                    if thetaSymm > 2*np.pi*symm:
                        thetaSymm = 2*np.pi*symm*2 - thetaSymm
                    x_cur = r_cur * np.cos(thetaSymm)
                    y_cur = r_cur * np.sin(thetaSymm)
                    new_array.append(dvec(x_cur, y_cur, z_cur))
                else:
                    x_cur, y_cur, z_cur = new_coords[k]
                    new_array.append(dvec(x_cur, y_cur, z_cur))
        
        dvec_new.vector().set_local(new_array)
        dvec_new.vector().apply("insert")
        results_cur.write(dvec_new, tNew[i])
        print(f"Done with time step {i} for file {j}")

 2024-10-28 19:22:40,135 smart.mesh - INFO - HDF5 mesh, "parent_mesh", successfully loaded from file: /root/shared/gitrepos/smart-nanopillars/model-files/../meshes/nanopillars_indent/nanopillars_indent2.8/spreadCell_mesh.h5! (mesh.py:245) 
 2024-10-28 19:22:40,138 smart.mesh - INFO - 0 subdomains successfully loaded from file: /root/shared/gitrepos/smart-nanopillars/model-files/../meshes/nanopillars_indent/nanopillars_indent2.8/spreadCell_mesh.h5! (mesh.py:258) 



Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Line)
Info    : [ 20%] Meshing curve 21 (Line)
Info    : [ 20%] Meshing curve 22 (Line)
Info    : [ 20%] Meshing curve 23 (Line)
Info    : [ 30%] Meshing curve 24 (Line)
